In [ ]:
import numpy as np
import time
import os
import h5py
import inspect
from tqdm import tqdm
import sys

import presto
from presto import lockin, utils
from presto.hardware import AdcFSample, AdcMode, DacFSample, DacMode

# Save attributes function
def save_attr(folder, file, sample_, myrun_, myrun_attrs_):
    if not os.path.isdir(folder):
        os.makedirs(folder)
    with h5py.File(os.path.join(folder, file), "a") as savefile:
        run_str = f"{myrun_}"
        # Assicurati che il gruppo esista
        if run_str not in savefile:
            savefile.create_group(run_str)
        for key, val in myrun_attrs_.items():
            savefile[run_str].attrs[key] = val
    print("Saved run attributes.")

# Save data function
def save_data(folder, file, sample_, myrun_, freq, usb_on_arr, usb_off_arr, index_dict):
    if not os.path.isdir(folder):
        os.makedirs(folder)

    # Open the save file (.hdf5) in append mode
    with h5py.File(os.path.join(folder, file), "a") as savefile:
        # Determine index number under run
        idx_amp = index_dict["idx_amp"]

        # String as handles
        freq_data_str = "{}/{}/freq comb".format(myrun_, str(idx_amp))
        usb_on_data_str = "{}/{}/USB ON".format(myrun_, str(idx_amp))
        usb_off_data_str = "{}/{}/USB OFF".format(myrun_, str(idx_amp))
        idx_amp_attrs_str = "{}/{}/".format(myrun_, str(idx_amp))

        # Write data to datasets
        savefile.create_dataset(freq_data_str, (np.shape(freq)),
                                dtype=float, data=freq)
        savefile.create_dataset(usb_on_data_str, (np.shape(usb_on_arr)),
                                dtype=complex, data=usb_on_arr)
        savefile.create_dataset(usb_off_data_str, (np.shape(usb_off_arr)),
                                dtype=complex, data=usb_off_arr)

        # Write index attributes
        savefile[idx_amp_attrs_str].attrs["pump amp"] = index_dict["pump_amp"]

        # Write dataset attributes
        savefile[freq_data_str].attrs["Unit"] = "Hz"
        savefile[usb_on_data_str].attrs["Unit"] = "fsu complex"
        savefile[usb_off_data_str].attrs["Unit"] = "fsu complex"


In [ ]:


Ym_str = time.strftime("%Y-%m")
meas_type   = 'Entanglement'
save_folder = r'I:/LKiPA-Data/{}/{}'.format(Ym_str, meas_type)
myrun       = time.strftime("%Y-%m-%d_%H_%M_%S")
save_file   = r"{}.hdf5".format(myrun)  

sample      = 'LKIPA'
atten       = 80
temperature = 0.0096

ADDRESS = '130.237.35.90'
PORT    = 42873
Box     = 'Presto DELTA'

input_port   = 5
output_port  = 8
pump_port    = 2
bias_port    = 2

DAC_CURRENT  = 32_000  # µA
bias_val     = 4.0   # FS  From calibration   4.00 V --> f0= 4.426348e9 Hz
# bias_val     = 4.50   # FS  From calibration   4.50 V --> f0= 4424855567.651496 Hz

# DSP e chunk
nr_sig_freqs = 95
if nr_sig_freqs % 2 == 0:
    nr_sig_freqs = nr_sig_freqs + 1  # Make it odd to have a tone at fNCO
_df           = 10e3
Npix         = 1_000_000   # 4_000_000
N_chunk      = 1_000
Navg         = 1


# Parametric pump
_f0 = 4.426348e9 # 
# _f0 = 4_424_855_567 #.651496
_fp_center   = 2*_f0
nr_sq_freqs  = 1
pump_max     = 0.1   #MAX TO USE 0.3 FS
pump_min     = 0.0
pump_steps   = 2
# amp_pump_arr = np.linspace(pump_min, pump_max, pump_steps) # np.array([0.05]) # 0.04*np.ones(40) 
amp_pump_arr = np.array([0.0, 0.1, 0.2, 0.3]) 
k_pmp=0
k_array      = np.array([k_pmp])
output_amp   = 0.0   # MAX 0.1 FS (not used now)


# Array for storing measurement data
usb_ON_arr  = np.zeros((Npix,
                        nr_sig_freqs), 
                        dtype=complex)
lsb_ON_arr = np.zeros_like(usb_ON_arr)
usb_OFF_arr = np.zeros_like(usb_ON_arr)
lsb_OFF_arr = np.zeros_like(usb_ON_arr)

# Propagation Phase Drift
input_phases = np.zeros(nr_sig_freqs)
dPhiDrift_df = 1.8869636024e-6  # rad/Hz - Measured Drift in phase per Hz 
# dPhiDrift_df = 1.8964498850e-6  # rad/Hz - Measured Drift in phase per Hz for df=1MHz

# Lock-in amplifier initialization
with lockin.Lockin(address=ADDRESS,
                   port=PORT,
                   adc_mode=AdcMode.Mixed,
                   adc_fsample=AdcFSample.G2,
                   dac_mode=[DacMode.Mixed04, DacMode.Mixed02, DacMode.Mixed02, DacMode.Mixed02],
                   dac_fsample=[DacFSample.G10, DacFSample.G6, DacFSample.G6, DacFSample.G6],
                   ) as lck:
    
    # Start timer
    t_start = time.strftime("%Y-%m-%d_%H_%M_%S")

    lck.hardware.set_dac_current(output_port, DAC_CURRENT)
    lck.hardware.set_dc_bias(bias_val, bias_port)
    lck.hardware.sleep(1.0, False)

    # Configure NCOs and frequency comb
    _, df= lck.tune(0, _df)
    
    fs_comb = np.arange(nr_sig_freqs) * df
    fNCO         = _f0 - int((nr_sig_freqs-1)/2)*df
    fNCO_pump    = 2*_f0

    lck.hardware.configure_mixer(freq=fNCO_pump,
                                 out_ports=pump_port,
                                 sync=False)

    lck.hardware.configure_mixer(freq=fNCO,
                                 in_ports=input_port,
                                 out_ports=output_port,
                                 sync=True)

    # Squeezing Pumps
    fp_comb   = np.array([0])

    # Set pumps phases
    phases_arr             = np.array([0])
    phases_q_arr           = phases_arr - np.pi/2

    lck.set_df(df)

    # Output group for Squeezing Pumps
    og_pump = lck.add_output_group(ports=pump_port, nr_freq=nr_sq_freqs)
    og_pump.set_frequencies(fp_comb)
    og_pump.set_phases(phases=phases_arr, phases_q=phases_q_arr)

    # Input group
    ig = lck.add_input_group(port=input_port, nr_freq=nr_sig_freqs)
    ig.set_frequencies(fs_comb)
    # Set input phases
    input_phases[0]=0.0
    for i in range(1, nr_sig_freqs):
        input_phases[i] = input_phases[i-1] - dPhiDrift_df * df  # Compensate the measured phase delay drift between modes.
    ig.set_phases(phases=input_phases.copy()) 

    lck.apply_settings()
    time.sleep(0.1)

    total_iters = len(amp_pump_arr) * (Npix // N_chunk)
    with tqdm(total=total_iters, ncols=80) as pbar:
        
        for amp_idx, amp_val in enumerate(amp_pump_arr):
            n_on =0
            for n in range(0, Npix, N_chunk):
                
                n_on +=N_chunk
                og_pump.set_amplitudes(amp_val * np.ones(nr_sq_freqs))
                
                lck.apply_settings()
                time.sleep(0.01)

                # Get lock-in packets (pixels) from the local buffer
                data = lck.get_pixels(N_chunk, summed=False, nsum=Navg)
                freqs, pixels_i, pixels_q = data[input_port]

                # Convert a measured IQ pair into a low/high sideband pair
                LSB, HSB = utils.untwist_downconversion(pixels_i, pixels_q)

                # Store data in array

                usb_ON_arr[n_on - N_chunk:n_on] = HSB
                lsb_ON_arr[n_on - N_chunk:n_on] = LSB

                # Update progress bar
                pbar.update(1)

            idx_dict = {
                "pump_amp":   amp_val,
                "idx_amp":    amp_idx,
            }
            if amp_idx == 0:
                usb_OFF_arr = usb_ON_arr.copy()
                lsb_OFF_arr = lsb_ON_arr.copy()
            save_data(save_folder, save_file, meas_type, myrun, fs_comb + fNCO, usb_ON_arr, usb_OFF_arr, idx_dict)


    og_pump.set_amplitudes(np.zeros(nr_sq_freqs))
    lck.apply_settings()
    lck.hardware.set_dc_bias(0.0, bias_port)

# Stop timer
t_end = time.strftime("%Y-%m-%d_%H_%M_%S")

myrun_attrs = {
    "Meas":        meas_type,
    "Instr":       Box,
    "T":           temperature,
    "Sample":      sample,
    "att":         atten,
    "fs_start":    fs_comb[0] + fNCO,
    "fs_stop":     fs_comb[-1] + fNCO,
    "df":          df,
    "nr_sig_freqs":nr_sig_freqs,
    "fp_comb":     fp_comb + fNCO_pump,
    "amp_pump":    amp_pump_arr,
    "phase_pump":  phases_arr,
    "DC bias":     bias_val,
    "Npixels":     Npix,
    "Nchunk":      N_chunk,
    "Naverages":   Navg,
    "t_start":     t_start,
    "t_end":       t_end,
}

save_attr(save_folder, save_file, meas_type, myrun, myrun_attrs)
print('Done.')


100%|███████████████████████████████████████| 4000/4000 [10:06<00:00,  6.60it/s]

Saved run attributes.
Done.


In [3]:
# print(df,fs_comb,len(fs_comb),nr_sig_freqs)